In [3]:
import gym;
import eplus_env;
import random
import statistics
import matplotlib.pyplot as plt
import pandas as pd

# Input: inTemps  = (r1,r2) where r1,r2 are a range of room temps (0,30)
#        outTemps = (r3,r4) where r3,r4 are a range of out temps (-17,35)
# Output: a dict with the cartesian product of both in and out temps
#         as a key and 0 as a value.
def getQTable(inTemps,outTemps):
  (r1,r2) = inTemps
  (r3,r4) = outTemps
  qList = []
  for i in range(r1,r2):
    for j in range(r3,r4):
        # Making a dict where the states (i,j) are the keys and the value 
        # is an array where the indices represent the actions (0 => off),
        # (1 => on = 30) respectively and the elements of the array are 
        # the Q-Value for each state and action (key and value).
        qList.append(((i,j),[0,0]))
  qDict = dict(qList)
  return qDict


def getFreqTable(inTemps,outTemps):
  (r1,r2) = inTemps
  (r3,r4) = outTemps
  freqList = []
  for i in range(r1,r2):
    for j in range(r3,r4):
        freqList.append(((i,j),0))
  freqDict = dict(freqList)
  return freqDict


max_energy = 11000
min_energy = 0
max_temp = 24
min_temp = 0


def normalize(x,x_max,x_min):

  if x < x_min:
    x = x_min
  elif x > x_max:
    x = x_max

  return (x_max - x) / (x_max - x_min)

def cubic(x, scale = - 0.1, translation = 0):
  return translation + (x**3) * scale

def getReward(sp,rt,E15):
  a = 0.5
  delta_sp = abs(sp - rt)
  norm_e = normalize(E15,max_energy,min_energy)
  norm_t = normalize(delta_sp,max_temp,min_temp)
  R = a * norm_e + (1-a) * norm_t
  return cubic(8 * R - 4, scale = -0.1, translation = 2)



sp = 30
inTemps = (0,31)
outTemps = (-17,35)
alpha = 0.6
gamma = 0.9
epsilon = 0.1

# making the initial Q-table
qTable = getQTable(inTemps,outTemps)

freqTable = getFreqTable(inTemps,outTemps)
env = gym.make('Eplus-test-v4')

cum_R_List = []

# Number of episodes
for i in range(500):
    cum_R = 0

    # Reset the env (creat the EnergyPlus subprocess)
    curSimTime, ob, isTerminal = env.reset()

    # get the initial in/out temps and the energy consumption
    state = (round(ob[8]),round(ob[0]))
    energyC = ob[14] # ???
    while not isTerminal:
      if random.uniform(0,1) < epsilon:
        choice = random.randint(0,1)
      else:
        # a list of qValues indexed by actions
        actionsList = qTable[state]
        choice = actionsList.index(max(actionsList))

      # (0 = off) (1 = on,30)
      if choice:
        action = [sp,sp]
      else:
        action = [0,0]

      curSimTime, ob, isTerminal = env.step(action)
      nextState = (round(ob[8]),round(ob[0]))

      # get the energy consumption for the last 15 mins
      energyC = abs(energyC - ob[14])
      reward = getReward(sp,ob[8],energyC)
      cum_R += reward

      oldQValue = qTable[state][choice]
      nextQMax = max(qTable[nextState])
      # update the Q - Value
      newQValue = oldQValue + alpha * (reward + gamma * nextQMax - oldQValue)
      qTable[state][choice] = newQValue
      # ble ate ice
      freqTable[state] += 1

      state = nextState
      # ate next ate (زي المفجوع)
    cum_R_List.append(cum_R)

# Safe termination of the environment after use.
env.end_env()

# bar chart to see the frequency of each state
#pd.DataFrame([freqTable]).T.plot.bar()

sorted_dict = {}
sorted_keys = sorted(freqTable, key=freqTable.get)  # [1, 3, 2]

for w in sorted_keys:
    sorted_dict[w] = freqTable[w]

print(sorted_dict)

# u late i u ate


# plot learning rate

[2021-07-14 16:31:19,663] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:31:19,663] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:31:19,663] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:31:19,666] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run1
[2021-07-14 16:31:19,666] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run1
[2021-07-14 16:31:19,666] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run1
[2021-07-14 16:31:48,783] EPLUS_ENV_Eplus-test-v4_MainThread_

[2021-07-14 16:34:25,691] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run7
[2021-07-14 16:34:25,691] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run7
[2021-07-14 16:34:57,222] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 16:34:57,222] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 16:34:57,222] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 16:34:57,224] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:34:57,224] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:34:57,224] EPLUS_ENV

[2021-07-14 16:38:13,714] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 16:38:13,718] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:38:13,718] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:38:13,718] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:38:13,725] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run14
[2021-07-14 16:38:13,725] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run14
[2021-07-14 16:38:13,725] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulat

[2021-07-14 16:41:40,735] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run20
[2021-07-14 16:41:40,735] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run20
[2021-07-14 16:42:17,644] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 16:42:17,644] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 16:42:17,644] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 16:42:17,648] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:42:17,648] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:42:17,648] EPLUS_E

[2021-07-14 16:46:03,472] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 16:46:03,472] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 16:46:03,475] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:46:03,475] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:46:03,475] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 16:46:03,485] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run27
[2021-07-14 16:46:03,485] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run27
[2021-07-14 16:46:03,485] EPL

...

[2021-07-14 20:03:33,541] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 20:03:33,541] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 20:03:33,543] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 20:03:33,543] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 20:03:33,543] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 20:03:33,550] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run477
[2021-07-14 20:03:33,550] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run477
[2021-07-14 20:03:33,550] E

[2021-07-14 20:05:48,180] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run483
[2021-07-14 20:05:48,180] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run483
[2021-07-14 20:05:48,180] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run483
[2021-07-14 20:06:09,612] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 20:06:09,612] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 20:06:09,612] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 20:06:09,615] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT I

[2021-07-14 20:08:24,103] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 20:08:24,103] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 20:08:24,103] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 20:08:24,105] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 20:08:24,105] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 20:08:24,105] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 20:08:24,112] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run490
[2021-07-14 20:08:24,112] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/

[2021-07-14 20:10:38,553] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Creating EnergyPlus simulation environment...
[2021-07-14 20:10:38,561] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run496
[2021-07-14 20:10:38,561] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run496
[2021-07-14 20:10:38,561] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:EnergyPlus working directory is in /home/abdalla/Desktop/QSIURP/Simulations/Eplus-env-Eplus-test-v4-res62/Eplus-env-sub_run496
[2021-07-14 20:11:00,324] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 20:11:00,324] EPLUS_ENV_Eplus-test-v4_MainThread_ROOT INFO:Last EnergyPlus process has been closed. 
[2021-07-14 20:11:00,324] EPLUS_ENV_Eplus-test-v4_MainThread_RO

{(0, -17): 0, (0, -16): 0, (0, -15): 0, (0, -14): 0, (0, -13): 0, (0, -12): 0, (0, -11): 0, (0, -10): 0, (0, -9): 0, (0, -8): 0, (0, -7): 0, (0, -6): 0, (0, -5): 0, (0, -4): 0, (0, -3): 0, (0, -2): 0, (0, -1): 0, (0, 0): 0, (0, 1): 0, (0, 2): 0, (0, 3): 0, (0, 4): 0, (0, 5): 0, (0, 6): 0, (0, 7): 0, (0, 8): 0, (0, 9): 0, (0, 10): 0, (0, 11): 0, (0, 12): 0, (0, 13): 0, (0, 14): 0, (0, 15): 0, (0, 16): 0, (0, 17): 0, (0, 18): 0, (0, 19): 0, (0, 20): 0, (0, 21): 0, (0, 22): 0, (0, 23): 0, (0, 24): 0, (0, 25): 0, (0, 26): 0, (0, 27): 0, (0, 28): 0, (0, 29): 0, (0, 30): 0, (0, 31): 0, (0, 32): 0, (0, 33): 0, (0, 34): 0, (1, -16): 0, (1, -15): 0, (1, -14): 0, (1, -13): 0, (1, -12): 0, (1, -11): 0, (1, -10): 0, (1, -9): 0, (1, -8): 0, (1, -7): 0, (1, -6): 0, (1, -5): 0, (1, -4): 0, (1, -3): 0, (1, -2): 0, (1, -1): 0, (1, 0): 0, (1, 1): 0, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): 0, (1, 6): 0, (1, 7): 0, (1, 8): 0, (1, 9): 0, (1, 10): 0, (1, 11): 0, (1, 12): 0, (1, 13): 0, (1, 14): 0, (1, 15):